In [1]:
from sklearn import random_projection, cross_validation, datasets, grid_search, linear_model, metrics, manifold, preprocessing
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from collections import  Counter
import numpy as np
import pandas as pd
from sklearn.decomposition import RandomizedPCA
from sklearn import manifold
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.decomposition import TruncatedSVD, PCA
import xgboost as xgb
%pylab inline

/home/tosha/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/tosha/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


In [2]:
list_df = [pd.read_csv('dataframe/genres_13.csv'), pd.read_csv('dataframe/genres_20.csv'),
           pd.read_csv('dataframe/genres_30.csv'), pd.read_csv('dataframe/genres_40.csv'), 
           pd.read_csv('dataframe/genres_50.csv')]
for i in range(5):
    list_df[i].loc[:, 'm1':] = preprocessing.normalize(list_df[i].loc[:, 'm1':])
    list_df[i] = list_df[i].sample(frac=1)

In [3]:
def search_abnom(dataframe, label, g, nu):
    svm_ = svm.OneClassSVM(gamma=g, nu=nu)
    svm_.fit(dataframe[dataframe['tag'] == label].loc[:, 'm1':])
    l = svm_.predict(dataframe[dataframe['tag'] == label].loc[:, 'm1':])
    return l

labels = ['hiphop', 'classical', 'jazz', 'rock', 'metal', 'synth', 'pop', 'blues']
for i in range(len(list_df)):
    for label in labels:
        svm_label = search_abnom(list_df[i], label, 2, 0.15).tolist()
        list_df[i][list_df[i]['tag'] == label] = list_df[i][list_df[i]['tag'] == label][np.array(svm_label) == 1]
        list_df[i] = list_df[i].dropna()

In [7]:
xgbc = xgb.XGBClassifier(silent=False, nthread=4, max_depth=2, 
                              n_estimators=8000, subsample=0.5, learning_rate=0.01)
# train_data, test_data, train_labels, test_labels = cross_validation.train_test_split(list_df[3].loc[:, 'm1':], list_df[3]['tag'], 
#                                                                                      test_size = 0.2)
# xgbc.fit(train_data, train_labels)
# print(metrics.classification_report(test_labels, xgbc.predict(test_data)))

In [8]:
list_rps = []
for i in range(5):
    ridge_scoring2 = cross_validation.cross_val_score(xgbc, list_df[i].loc[:, 'm1':], list_df[i]['tag'], 
                                                          scoring='f1_weighted', cv = 5)
    list_rps.append(ridge_scoring2)

m = np.transpose(np.array([list_rps]).mean(axis=2))
s = np.transpose(np.array([list_rps]).std(axis=2))
                 
figure(figsize=(6,5))
plt.imshow(m)
plt.colorbar()
plt.yticks(np.arange(5), ('13', '20', '30', '40', '50'))
plt.xticks([0], ('mean',))
show()

figure(figsize=(6,5))
plt.imshow(s)
plt.colorbar()
plt.yticks(np.arange(5), ('13', '20', '30', '40', '50'))
plt.xticks([0], ('std',))
show()

KeyboardInterrupt: 

In [17]:
xgbc.fit(list_df[3].loc[:, 'm1':], list_df[3].loc[:, 'tag'])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.03, max_delta_step=0,
       max_depth=2, min_child_weight=1, missing=None, n_estimators=3800,
       n_jobs=1, nthread=4, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=False, subsample=0.5)

In [21]:
fi = xgbc.get_booster().get_fscore()
fi_l = []
for f in list_df[2].columns[1:]:
    fi_l.append(fi[f])

In [22]:
list_numb = [0.5,  1,  1.5,  1.9]
list_rps = []
for j in range(4):
    data_more_med = list_df[3].loc[:, fi[3]>(np.median(fi[3])*list_numb[j])]
    ridge_scoring2 = cross_validation.cross_val_score(xgbc, data_more_med.loc[:, 'm1':], data_more_med.loc[:, 'tag'], 
                                                              scoring='f1_weighted', cv = 5)
    #         print('Ridge mean:{}, max:{}, min:{}, std:{}'.format(ridge_scoring2.mean(), ridge_scoring2.max(),
    #                                                              ridge_scoring2.min(), ridge_scoring2.std()))
    list_rps.append(ridge_scoring2)

m = np.transpose(np.array([list_rps]).mean(axis=2))
s = np.transpose(np.array([list_rps]).std(axis=2))
    
figure(figsize=(6,5))
plt.imshow(m)
plt.colorbar()
plt.yticks(np.arange(4), ('0.5',  '1',  '1.5',  '1.9'))
plt.xticks([0], ('mean',))
show()

figure(figsize=(6,5))
plt.imshow(s)
plt.colorbar()
plt.yticks(np.arange(4), ('0.5', '1',  '1.5',  '1.9'))
plt.xticks([0], ('std',))
show()

KeyError: 3